# Séance 3 : compléments sur API

Open Data Soft

Aide : https://help.opendatasoft.com/apis/ods-explore-v2/explore_v2.1.html

## API Vélib

https://opendata.paris.fr/explore/dataset/velib-disponibilite-en-temps-reel/api/



In [89]:
# Premier essai -> on obtient que 10 résultats
import requests

url_base = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records"
resultat = requests.get(url_base).json()
resultat

{'total_count': 1484,
 'results': [{'stationcode': '14014',
   'name': 'Jourdan - Stade Charléty',
   'is_installed': 'OUI',
   'capacity': 60,
   'numdocksavailable': 57,
   'numbikesavailable': 3,
   'mechanical': 1,
   'ebike': 2,
   'is_renting': 'OUI',
   'is_returning': 'OUI',
   'duedate': '2024-12-04T06:17:16+00:00',
   'coordonnees_geo': {'lon': 2.3433353751898, 'lat': 48.819428333369},
   'nom_arrondissement_communes': 'Paris',
   'code_insee_commune': '75056'},
  {'stationcode': '8026',
   'name': 'Messine - Place Du Pérou',
   'is_installed': 'OUI',
   'capacity': 12,
   'numdocksavailable': 3,
   'numbikesavailable': 8,
   'mechanical': 3,
   'ebike': 5,
   'is_renting': 'OUI',
   'is_returning': 'OUI',
   'duedate': '2024-12-04T06:16:31+00:00',
   'coordonnees_geo': {'lon': 2.315508019010038, 'lat': 48.875448033960744},
   'nom_arrondissement_communes': 'Paris',
   'code_insee_commune': '75056'},
  {'stationcode': '6003',
   'name': 'Saint-Sulpice',
   'is_installed': 'OU

In [90]:
# avec un paramètre limit = au max
requests.get(url_base + "?limit=" + str(resultat["total_count"])).json()

{'error_code': 'InvalidRESTParameterError',
 'message': 'Invalid value for limit API parameter: 1484 was found but -1 <= limit <= 100 is expected.'}

In [77]:
# il faut faire une boucle pour avoir tous les résultats
resultat_final = []
for i in range(0, resultat["total_count"], 100):
    temp = requests.get(url_base + "?limit=100&offset=" + str(i)).json()
    resultat_final += temp["results"]

# on trouve bien nos 1484 résultats
len(resultat_final)

1484

In [78]:
# que l'on peut transformer en DataFrame pandas
import pandas

data = pandas.DataFrame(resultat_final)
data

,stationcode,name,is_installed,capacity,numdocksavailable,numbikesavailable,mechanical,ebike,is_renting,is_returning,duedate,coordonnees_geo,nom_arrondissement_communes,code_insee_commune
0,8026,Messine - Place Du Pérou,OUI,12,3,8,3,5,OUI,OUI,2024-12-04T06:16:31+00:00,"{'lon': 2.315508019010038, 'lat': 48.875448033...",Paris,75056
1,13007,Le Brun - Gobelins,OUI,48,19,29,15,14,OUI,OUI,2024-12-04T06:15:34+00:00,"{'lon': 2.3534681351338, 'lat': 48.835092787824}",Paris,75056
2,6003,Saint-Sulpice,OUI,21,7,12,7,5,OUI,OUI,2024-12-04T06:19:40+00:00,"{'lon': 2.3308077827095985, 'lat': 48.85165383...",Paris,75056
3,5110,Lacépède - Monge,OUI,23,10,12,11,1,OUI,OUI,2024-12-04T06:18:23+00:00,"{'lon': 2.3519663885235786, 'lat': 48.84389286...",Paris,75056
4,6108,Saint-Romain - Cherche-Midi,OUI,17,11,6,5,1,OUI,OUI,2024-12-04T06:14:35+00:00,"{'lon': 2.321374788880348, 'lat': 48.847081590...",Paris,75056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,21021,Enfants du Paradis - Peupliers,OUI,40,24,21,18,3,OUI,OUI,2024-12-04T06:19:37+00:00,"{'lon': 2.2571459412575, 'lat': 48.833122686454}",Boulogne-Billancourt,92012
1480,9104,Caumartin - Provence,OUI,22,10,10,4,6,OUI,OUI,2024-12-04T06:18:45+00:00,"{'lon': 2.3284685611724854, 'lat': 48.87442277...",Paris,75056
1481,33019,Madeleine Vionnet,OUI,34,26,8,4,4,OUI,OUI,2024-12-04T06:16:04+00:00,"{'lon': 2.3740895837545, 'lat': 48.902519074064}",Aubervilliers,93001
1482,40002,Bleuets - Bordières,NON,1,1,0,0,0,NON,NON,2024-08-27T09:20:23+00:00,"{'lon': 2.4543687905029, 'lat': 48.802117531472}",Créteil,94028


In [79]:
# On a quelques stations en double, voire triple
data.groupby("stationcode").size().sort_values(ascending=False)

stationcode
13125    3
13019    3
15008    3
15001    3
2101     3
        ..
11201    1
9037     1
9038     1
12001    1
1002     1
Length: 1261, dtype: int64

In [80]:
data.columns

Index(['stationcode', 'name', 'is_installed', 'capacity', 'numdocksavailable',
       'numbikesavailable', 'mechanical', 'ebike', 'is_renting',
       'is_returning', 'duedate', 'coordonnees_geo',
       'nom_arrondissement_communes', 'code_insee_commune'],
      dtype='object')

In [81]:
# On peut ne sélectionner que certaines variables (utlis si le résultat final sera bcp trop gros en taille)
requests.get(url_base + "?select=stationcode,name").json()

{'total_count': 1484,
 'results': [{'stationcode': '14014', 'name': 'Jourdan - Stade Charléty'},
  {'stationcode': '8026', 'name': 'Messine - Place Du Pérou'},
  {'stationcode': '6003', 'name': 'Saint-Sulpice'},
  {'stationcode': '7002', 'name': 'Vaneau - Sèvres'},
  {'stationcode': '5110', 'name': 'Lacépède - Monge'},
  {'stationcode': '6108', 'name': 'Saint-Romain - Cherche-Midi'},
  {'stationcode': '40011', 'name': 'Bas du Mont-Mesly'},
  {'stationcode': '21010', 'name': 'Silly - Galliéni'},
  {'stationcode': '25006', 'name': 'Place Nelson Mandela'},
  {'stationcode': '30002', 'name': 'Jean Rostand - Paul Vaillant Couturier'}]}

In [82]:
# on peut même faire du SQL
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30").json()

{'total_count': 632,
 'results': [{'stationcode': '14014', 'capacity': 60},
  {'stationcode': '7002', 'capacity': 35},
  {'stationcode': '30002', 'capacity': 40},
  {'stationcode': '7003', 'capacity': 60},
  {'stationcode': '17026', 'capacity': 40},
  {'stationcode': '8050', 'capacity': 45},
  {'stationcode': '31024', 'capacity': 38},
  {'stationcode': '2022', 'capacity': 35},
  {'stationcode': '10161', 'capacity': 32},
  {'stationcode': '41303', 'capacity': 32}]}

In [83]:
# avec du tri
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30&order_by=capacity").json()

{'total_count': 632,
 'results': [{'stationcode': '5123', 'capacity': 31},
  {'stationcode': '22101', 'capacity': 31},
  {'stationcode': '9117', 'capacity': 31},
  {'stationcode': '17016', 'capacity': 31},
  {'stationcode': '20202', 'capacity': 31},
  {'stationcode': '8105', 'capacity': 31},
  {'stationcode': '10110', 'capacity': 31},
  {'stationcode': '21307', 'capacity': 31},
  {'stationcode': '22206', 'capacity': 31},
  {'stationcode': '16105', 'capacity': 31}]}

In [84]:
# décroissant c'est encore mieux
requests.get(url_base + "?select=stationcode,capacity&where=capacity>30&order_by=capacity DESC").json()

{'total_count': 632,
 'results': [{'stationcode': '7009', 'capacity': 76},
  {'stationcode': '15030', 'capacity': 74},
  {'stationcode': '15028', 'capacity': 71},
  {'stationcode': '5034', 'capacity': 69},
  {'stationcode': '12157', 'capacity': 68},
  {'stationcode': '32012', 'capacity': 68},
  {'stationcode': '16025', 'capacity': 68},
  {'stationcode': '12106', 'capacity': 68},
  {'stationcode': '1013', 'capacity': 67},
  {'stationcode': '15012', 'capacity': 67}]}

In [ ]:
# plus loin dans le WHERE - combinaison de clauses
requests.get(url_base + "?select=stationcode,capacity,is_renting&where=capacity>20 AND is_renting='NON'&limit=100").json()

{'total_count': 17,
 'results': [{'stationcode': '42207', 'capacity': 37, 'is_renting': 'NON'},
  {'stationcode': '10162', 'capacity': 24, 'is_renting': 'NON'},
  {'stationcode': '3005', 'capacity': 33, 'is_renting': 'NON'},
  {'stationcode': '18010', 'capacity': 26, 'is_renting': 'NON'},
  {'stationcode': '40014', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '40015', 'capacity': 24, 'is_renting': 'NON'},
  {'stationcode': '40004', 'capacity': 25, 'is_renting': 'NON'},
  {'stationcode': '40001', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '18203', 'capacity': 40, 'is_renting': 'NON'},
  {'stationcode': '44010', 'capacity': 30, 'is_renting': 'NON'},
  {'stationcode': '43003', 'capacity': 26, 'is_renting': 'NON'},
  {'stationcode': '15063', 'capacity': 23, 'is_renting': 'NON'},
  {'stationcode': '13048', 'capacity': 28, 'is_renting': 'NON'},
  {'stationcode': '18027', 'capacity': 30, 'is_renting': 'NON'},
  {'stationcode': '40009', 'capacity': 30, 'is_renting': 'NO

In [133]:
# qui commence
requests.get(url_base + "?select=stationcode,name&where=startswith(name,'Exelmans')").json()

{'total_count': 2,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16040', 'name': 'Exelmans - Michel-Ange'}]}

In [136]:
# qui contient
requests.get(url_base + "?select=stationcode,name&where=suggest(name,'Versailles')").json()

{'total_count': 3,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16041', 'name': 'Versailles - Claude Terrasse'},
  {'stationcode': '15203', 'name': 'Porte de Versailles'}]}

In [ ]:
# qui est égal à 
requests.get(url_base + "?select=stationcode,name&where=search(name,'Exelmans - Versailles')").json()

{'total_count': 1,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'}]}

In [ ]:
# clause IN
requests.get(url_base + "?select=stationcode,nom_arrondissement_communes&where=nom_arrondissement_communes IN ('Clichy','Colombes')").json()

{'total_count': 18,
 'results': [{'stationcode': '27002',
   'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27004', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27005', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27001', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '27006', 'nom_arrondissement_communes': 'Colombes'},
  {'stationcode': '21109', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21105', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21108', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21120', 'nom_arrondissement_communes': 'Clichy'},
  {'stationcode': '21112', 'nom_arrondissement_communes': 'Clichy'}]}

In [88]:
# Les stations à moins de 800m de l'IUT
requests.get(url_base + "?select=stationcode,name&where=within_distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)', 800m)").json()

{'total_count': 15,
 'results': [{'stationcode': '15104',
   'name': 'Hôpital Européen Georges Pompidou'},
  {'stationcode': '16032', 'name': "Eglise d'Auteuil"},
  {'stationcode': '15059', 'name': 'Parc André Citroën'},
  {'stationcode': '16037', 'name': 'Molitor - Michel-Ange'},
  {'stationcode': '15058', 'name': 'Place du Moulin de Javel'},
  {'stationcode': '15103', 'name': 'Cauchy - Cévennes'},
  {'stationcode': '15068',
   'name': 'Général Martial Valin - Pont du Garigliano'},
  {'stationcode': '16118', 'name': 'Michel-Ange - Parent de Rosan'},
  {'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16038', 'name': 'Molitor - Chardon-Lagache'}]}

In [141]:
# idem mais avec la distance réelle
requests.get(url_base + "?select=stationcode,name,distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)') as distance&where=within_distance(coordonnees_geo, geom'POINT(2.267888940737877 48.84197963193564)', 800m)").json()

{'total_count': 15,
 'results': [{'stationcode': '15068',
   'name': 'Général Martial Valin - Pont du Garigliano',
   'distance': 458.73543527109746},
  {'stationcode': '16039',
   'name': 'Exelmans - Versailles',
   'distance': 279.35537451504047},
  {'stationcode': '15104',
   'name': 'Hôpital Européen Georges Pompidou',
   'distance': 725.9808199551429},
  {'stationcode': '15059',
   'name': 'Parc André Citroën',
   'distance': 759.235801861452},
  {'stationcode': '16041',
   'name': 'Versailles - Claude Terrasse',
   'distance': 355.48687528576613},
  {'stationcode': '16037',
   'name': 'Molitor - Michel-Ange',
   'distance': 552.7017995780018},
  {'stationcode': '15060',
   'name': 'Place de la Montagne du Goulet',
   'distance': 742.8498846967603},
  {'stationcode': '15058',
   'name': 'Place du Moulin de Javel',
   'distance': 335.3045956887677},
  {'stationcode': '16118',
   'name': 'Michel-Ange - Parent de Rosan',
   'distance': 710.8022353937183},
  {'stationcode': '16038',
 

In [ ]:
# clause in_bbox 
requests.get(url_base + "?select=stationcode,name&where=in_bbox(coordonnees_geo,48.84,2.26,48.85,2.27)").json()

{'total_count': 8,
 'results': [{'stationcode': '16039', 'name': 'Exelmans - Versailles'},
  {'stationcode': '16041', 'name': 'Versailles - Claude Terrasse'},
  {'stationcode': '16037', 'name': 'Molitor - Michel-Ange'},
  {'stationcode': '16038', 'name': 'Molitor - Chardon-Lagache'},
  {'stationcode': '16034', 'name': "Porte d'Auteuil"},
  {'stationcode': '16032', 'name': "Eglise d'Auteuil"},
  {'stationcode': '16116', 'name': 'George Sand - Jean de La Fontaine'},
  {'stationcode': '16033', 'name': "Marché d'Auteuil"}]}

In [ ]:
# clause in_bbox avec la clause NOT
requests.get(url_base + "?select=stationcode,name&where=not(in_bbox(coordonnees_geo,48.84,2.26,48.85,2.27))").json()

{'total_count': 1476,
 'results': [{'stationcode': '44015', 'name': "Rouget de L'isle - Watteau"},
  {'stationcode': '9020', 'name': 'Toudouze - Clauzel'},
  {'stationcode': '14111', 'name': 'Cassini - Denfert-Rochereau'},
  {'stationcode': '11104', 'name': 'Charonne - Robert et Sonia Delaunay'},
  {'stationcode': '13007', 'name': 'Le Brun - Gobelins'},
  {'stationcode': '6003', 'name': 'Saint-Sulpice'},
  {'stationcode': '5110', 'name': 'Lacépède - Monge'},
  {'stationcode': '40011', 'name': 'Bas du Mont-Mesly'},
  {'stationcode': '21010', 'name': 'Silly - Galliéni'},
  {'stationcode': '7003', 'name': 'Square Boucicaut'}]}

In [85]:
# p'tit group by ?
requests.get(url_base + "?select=nom_arrondissement_communes,count(*),sum(capacity)&group_by=nom_arrondissement_communes").json()

{'results': [{'nom_arrondissement_communes': 'Alfortville',
   'count(*)': 5,
   'sum(capacity)': 122},
  {'nom_arrondissement_communes': 'Arcueil',
   'count(*)': 4,
   'sum(capacity)': 118},
  {'nom_arrondissement_communes': 'Argenteuil',
   'count(*)': 7,
   'sum(capacity)': 225},
  {'nom_arrondissement_communes': 'Asnières-sur-Seine',
   'count(*)': 13,
   'sum(capacity)': 329},
  {'nom_arrondissement_communes': 'Aubervilliers',
   'count(*)': 13,
   'sum(capacity)': 404},
  {'nom_arrondissement_communes': 'Bagneux',
   'count(*)': 6,
   'sum(capacity)': 155},
  {'nom_arrondissement_communes': 'Bagnolet',
   'count(*)': 8,
   'sum(capacity)': 249},
  {'nom_arrondissement_communes': 'Bobigny',
   'count(*)': 5,
   'sum(capacity)': 132},
  {'nom_arrondissement_communes': 'Bois-Colombes',
   'count(*)': 2,
   'sum(capacity)': 60},
  {'nom_arrondissement_communes': 'Boulogne-Billancourt',
   'count(*)': 29,
   'sum(capacity)': 877},
  {'nom_arrondissement_communes': 'Bourg-la-Reine',
 

In [86]:
# et avec de l'ordre (et un renomage de variables)
requests.get(url_base + "?select=nom_arrondissement_communes,count(*) as nb_stations,sum(capacity)&group_by=nom_arrondissement_communes&order_by=nb_stations DESC").json()

{'results': [{'nom_arrondissement_communes': 'Paris',
   'nb_stations': 991,
   'sum(capacity)': 31893},
  {'nom_arrondissement_communes': 'Boulogne-Billancourt',
   'nb_stations': 29,
   'sum(capacity)': 877},
  {'nom_arrondissement_communes': 'Montreuil',
   'nb_stations': 23,
   'sum(capacity)': 787},
  {'nom_arrondissement_communes': 'Issy-les-Moulineaux',
   'nb_stations': 22,
   'sum(capacity)': 728},
  {'nom_arrondissement_communes': 'Pantin',
   'nb_stations': 21,
   'sum(capacity)': 565},
  {'nom_arrondissement_communes': 'Saint-Denis',
   'nb_stations': 19,
   'sum(capacity)': 666},
  {'nom_arrondissement_communes': 'Ivry-sur-Seine',
   'nb_stations': 18,
   'sum(capacity)': 590},
  {'nom_arrondissement_communes': 'Vitry-sur-Seine',
   'nb_stations': 16,
   'sum(capacity)': 425},
  {'nom_arrondissement_communes': 'Créteil',
   'nb_stations': 14,
   'sum(capacity)': 218},
  {'nom_arrondissement_communes': 'Asnières-sur-Seine',
   'nb_stations': 13,
   'sum(capacity)': 329},
  

## Fonctions spéciales

- RANGE : découpage quanti (quelconque mais aussi potentiellement date)
- GEO_CLUSTER : découpage selon un niveau de zoom

In [ ]:
# Découpage d'une quanti en intervalles de même taille
requests.get(url_base + "?select=count(*)&group_by=RANGE(capacity,15)").json()

{'results': [{'RANGE(capacity,15)': '[0, 14]',
   'stationcode': None,
   'count(*)': 43},
  {'RANGE(capacity,15)': '[15, 29]', 'stationcode': None, 'count(*)': 716},
  {'RANGE(capacity,15)': '[30, 44]', 'stationcode': None, 'count(*)': 509},
  {'RANGE(capacity,15)': '[45, 59]', 'stationcode': None, 'count(*)': 166},
  {'RANGE(capacity,15)': '[60, 74]', 'stationcode': None, 'count(*)': 49},
  {'RANGE(capacity,15)': '[75, 89]', 'stationcode': None, 'count(*)': 1}]}

In [ ]:
# Découpage d'une quanti en intervalles avec bornes indiquées (et renommage de la variable)
requests.get(url_base + "?select=count(*)&group_by=RANGE(capacity,*,15,20,25,30,40,*) as capacity").json()

{'results': [{'capacity': '[*, 14]', 'stationcode': None, 'count(*)': 43},
  {'capacity': '[15, 19]', 'stationcode': None, 'count(*)': 135},
  {'capacity': '[20, 24]', 'stationcode': None, 'count(*)': 303},
  {'capacity': '[25, 29]', 'stationcode': None, 'count(*)': 278},
  {'capacity': '[30, 39]', 'stationcode': None, 'count(*)': 407},
  {'capacity': '[40, *]', 'stationcode': None, 'count(*)': 318}]}

In [115]:
# Découpage en zone ?
requests.get(url_base + "?select=count(*)&group_by=GEO_CLUSTER(coordonnees_geo,10)").json()

{'results': [{'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.87072540944228,
     'lon': 2.364082421900835}},
   'count(*)': 613},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.83896064189873,
     'lon': 2.4141708750213535}},
   'count(*)': 174},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.890181979141765,
     'lon': 2.308858635858065}},
   'count(*)': 205},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.912563062357634,
     'lon': 2.2608207631856208}},
   'count(*)': 33},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.84262545383535,
     'lon': 2.2577943018062374}},
   'count(*)': 136},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.947931905942305,
     'lon': 2.243483570803489}},
   'count(*)': 7},
  {'GEO_CLUSTER(coordonnees_geo,10)': {'cluster_centroid': {'lat': 48.87852969300002,
     'lon': 2.194462900981307}},
   'count(*)': 15},
  {'GEO

## TRAVAIL A FAIRE

A partir de l'API : https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/information/?sort=date
Observation météorologique historiques France (SYNOP)

In [142]:
url_base = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/donnees-synop-essentielles-omm/records"
requests.get(url_base).json()


{'total_count': 2531637,
 'results': [{'numer_sta': '78925',
   'date': '2017-04-06T15:00:00+00:00',
   'pmer': 101830,
   'tend': -10.0,
   'cod_tend': '8',
   'dd': 110,
   'ff': 8.3,
   't': 303.45,
   'td': 294.25,
   'u': 58,
   'vv': 53550.0,
   'ww': None,
   'w1': None,
   'w2': None,
   'n': None,
   'nbas': None,
   'hbas': None,
   'cl': None,
   'cm': None,
   'ch': None,
   'pres': 101740.0,
   'niv_bar': None,
   'geop': None,
   'tend24': -10.0,
   'tn12': None,
   'tn24': None,
   'tx12': None,
   'tx24': None,
   'tminsol': None,
   'sw': None,
   'tw': None,
   'raf10': None,
   'rafper': None,
   'per': None,
   'etat_sol': None,
   'ht_neige': None,
   'ssfrai': None,
   'perssfrai': None,
   'rr1': 0.0,
   'rr3': 0.0,
   'rr6': 0.0,
   'rr12': -0.1,
   'rr24': -0.1,
   'phenspe1': None,
   'phenspe2': None,
   'phenspe3': None,
   'phenspe4': None,
   'nnuage1': None,
   'ctype1': None,
   'hnuage1': None,
   'nnuage2': None,
   'ctype2': None,
   'hnuage2': None,


In [144]:
pandas.DataFrame(requests.get(url_base).json()["results"])

,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,u,...,altitude,libgeo,codegeo,nom_epci,code_epci,nom_dept,code_dep,nom_reg,code_reg,mois_de_l_annee
0,78925,2017-04-06T15:00:00+00:00,101830.0,-10.0,8,110,8.3,303.45,294.25,58,...,3,Le Lamentin,97213,CA du Centre de la Martinique,249720061,Martinique,972,Martinique,02,4
1,07110,2017-04-06T18:00:00+00:00,102910.0,-40.0,6,30,4.1,284.55,279.25,70,...,94,Guipavas,29075,Brest Métropole,242900314,Finistère,29,Bretagne,53,4
2,07481,2017-04-06T18:00:00+00:00,102170.0,80.0,3,340,6.6,285.75,275.55,50,...,235,Colombier-Saugnieu,69299,CC de l'Est Lyonnais (CCEL),246900575,Rhône,69,Auvergne-Rhône-Alpes,84,4
3,07661,2017-04-06T18:00:00+00:00,101610.0,110.0,3,200,3.2,286.85,284.75,87,...,115,Saint-Mandrier-sur-Mer,83153,Métropole Toulon-Provence-Méditerranée,248300543,Var,83,Provence-Alpes-Côte d'Azur,93,4
4,07761,2017-04-06T18:00:00+00:00,101540.0,100.0,3,170,1.2,288.65,282.35,66,...,5,Ajaccio,2a004,CA du Pays Ajaccien,242010056,Corse-du-Sud,2a,Corse,94,4
5,07130,2017-04-06T21:00:00+00:00,102900.0,80.0,2,360,3.5,282.25,279.35,82,...,36,Saint-Jacques-de-la-Lande,35281,Rennes Métropole,243500139,Ille-et-Vilaine,35,Bretagne,53,4
6,07591,2017-04-06T21:00:00+00:00,NaN,300.0,1,360,0.8,281.35,275.15,65,...,871,Embrun,05046,CC Serre-Ponçon,200067742,Hautes-Alpes,05,Provence-Alpes-Côte d'Azur,93,4
7,61980,2017-04-06T21:00:00+00:00,101620.0,-50.0,8,110,4.7,298.35,292.75,71,...,8,Sainte-Marie,97418,CA Intercommunale du Nord de la Réunion (CINOR),249740119,La Réunion,974,La Réunion,04,4
8,61976,2017-04-07T00:00:00+00:00,101250.0,10.0,None,110,7.4,300.15,295.15,74,...,7,None,None,None,None,None,None,None,None,4
9,07280,2017-04-07T03:00:00+00:00,102600.0,-70.0,6,360,3.6,277.35,273.65,77,...,219,Ouges,21473,Dijon Métropole,242100410,Côte-d'Or,21,Bourgogne-Franche-Comté,27,4
